In [1]:
from __future__ import division
import sys
import numpy as np
import pandas as pd
def convert():
    """RFH feature"""
    dataPath= "data/human.fasta"
    lines=open(dataPath).readlines()
    outputPath="data/human_GAC_RFH.csv"
    classifier_type='U'#sys.argv[3]
    finally_text = open(outputPath, 'w')
    finnaly_lines=""
    for line in lines:
        if line.strip()=="":continue
        if line.strip()[0] in ['A','G','C',classifier_type]:
            position_mark=0
            count_AGCT=[0,0,0,0]
            temp = ""
            one_line=list(line.strip())
            for i,x in enumerate(one_line):
                if i!=9 and i!=10 and i!=11:
                    position_mark+=1
                    if x=="A" or x=="G":temp+="1,"
                    else:temp+="0,"
                    if x=="A" or x==classifier_type:temp+="1,"
                    else:temp+="0,"
                    if x == "A" or x == "C":temp+="1,"
                    else:temp+="0,"
                    if x == "A":
                        count_AGCT[0] += 1
                        temp +=str(round(count_AGCT[0] / position_mark*1.0,2))
                        temp+=','
                    elif x == "G":
                        count_AGCT[1] += 1
                        temp +=str(round(count_AGCT[1] / position_mark*1.0,2))
                        temp += ','
                    elif x == "C":
                        count_AGCT[2] += 1
                        temp +=str(round(count_AGCT[2] / position_mark*1.0,2))
                        temp += ','
                    elif x == classifier_type:
                        count_AGCT[3] += 1
                        temp +=str(round(count_AGCT[3] / position_mark*1.0,2))
                        temp += ','

            finnaly_lines+=((temp[:len(temp)-1])+'\n')
            #finally_text.write(temp+'\n')
    finally_text.writelines(finnaly_lines)
    finally_text.close()
convert()
data=pd.read_csv("data/human_GAC_RFH.csv",header=None,index_col=False)
data=data.values
print data.shape

(2260, 152)


In [2]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost.sklearn import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
import csv
import re
import time
import os
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
path="data/"
inputname="human.fasta"
outputname="human_GAC_RFH"

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [8]:
"""
    cross validation
"""
echo_all=10
for echo_index in xrange(1,echo_all+1):
    print "start"
    startTime=time.time()
    datapath =path+outputname+'.csv'
    train_data = pd.read_csv(datapath, header=None, index_col=None)
    X = np.array(train_data)
    Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
    Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
    Y.extend(Y2)
    Y = np.array(Y)
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=echo_index, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
    ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata
    print X.shape[1]
    endTime=time.time()
    elapsedTime=endTime-startTime
    print  "echo:",echo_index," elapsed time:",elapsedTime
    print "end"

start
0.900884955752
[[['svmC:2.82842712475gamma:0.03125', 0.90088495575221239, 0.9934640522875817, 0.8070796460176991, 0.8070796460176991, 0.9946902654867257, 0.8959878723321414, 0.89062499999999989, 0.89062499999999989, 0.8162639174737063, '0.9009', 912.0, 218.0, 6.0, 1124.0, 1130.0, 1130.0]]]
152
echo: 1  elapsed time: 828.228752136
end
start
0.900884955752
[[['svmC:2.82842712475gamma:0.03125', 0.90088495575221239, 0.9934640522875817, 0.8070796460176991, 0.8070796460176991, 0.9946902654867257, 0.8959878723321414, 0.89062499999999989, 0.89062499999999989, 0.8162639174737063, '0.9009', 912.0, 218.0, 6.0, 1124.0, 1130.0, 1130.0]]]
152
echo: 2  elapsed time: 420.426326036
end
start
0.900884955752
[[['svmC:2.82842712475gamma:0.03125', 0.90088495575221239, 0.9934640522875817, 0.8070796460176991, 0.8070796460176991, 0.9946902654867257, 0.8959878723321414, 0.89062499999999989, 0.89062499999999989, 0.8162639174737063, '0.9009', 912.0, 218.0, 6.0, 1124.0, 1130.0, 1130.0]]]
152
echo: 3  elapse